# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure ***MATERIALS_DIR*** points to the directory where you extracted the Zip file.
* Make sure all your paths are **relative to ** ***MATERIALS_DIR*** and **NOT hard-coded** in your code.

In [1]:
# imports
# Put all your imports here
#coding:utf-8
from whoosh import index, writing, scoring
from whoosh.scoring import Weighting
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
import os, os.path
import shutil
import nltk
from nltk.stem import *

In [2]:
MATERIALS_DIR = r"C:\DSS_Fall2017_Assign2"
#
# Put other path constants here
DOCUMENTS_DIR = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\documents")
INDEX_DIR = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\index1")
QUER_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\topics\gov.topics")
QRELS_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\qrels\gov.qrels")
OUTPUT_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\myres")
TREC_EVAL = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\trec_eval\trec_eval.exe")
INDEX_DIR2 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\index2")
OUTPUT_FILE2 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\myres2")
INDEX_DIR3 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\index3")
OUTPUT_FILE3 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\myres3")

In [3]:
'''
Initializing multiple analyzers here.
'''
# we start with basic tokenizer
tokenizer = RegexTokenizer()
#[token.text for token in tokenizer("We are going to do Text Analysis with whoosh.analysis")]

# we might want use stemming:
stmAnalyzer = RegexTokenizer() | StemFilter()
#[token.text for token in stmAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

# We probably want to lower-case it
# so we add LowercaseFilter
stmLwrAnalyzer = RegexTokenizer() | LowercaseFilter() | StemFilter()
#[token.text for token in stmLwrAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

# we probably want to ignore words like "we", "are", "with" when we index files
# so we add StopFilter to filter stop words
stmLwrStpAnalyzer = RegexTokenizer() | LowercaseFilter() | StopFilter() | StemFilter()
#[token.text for token in stmLwrStpAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

# we also probably want to break phrases like "whoosh.analysis" into "whoosh" and "analysis"
# so we add IntraWordFilter
stmLwrStpIntraAnalyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | StemFilter()
#[token.text for token in stmLwrStpIntraAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

In [4]:
# download required resources
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sx449_000\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
'''
Initialize CustomFilter class
'''
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
from whoosh.analysis import Filter
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [6]:
'''
Initialzie NLTK filters.
'''
# Example1: Whoosh filter for NLTK's LancasterStemmer
myFilter1 = RegexTokenizer() | CustomFilter(LancasterStemmer().stem)
# [token.text for token in myFilter1("We are going to do Text Analysis with whoosh.analysis")]

# Example2: Whoosh filter for NLTK's WordNetLemmatizer
myFilter2 = RegexTokenizer() | CustomFilter(WordNetLemmatizer().lemmatize)
# [token.text for token in myFilter2("We are going to do Text Analysis with whoosh.analysis")]

# Example3: Whoosh filter for NLTK's WordNetLemmatizer for verbs
myFilter3 = RegexTokenizer() | CustomFilter(WordNetLemmatizer().lemmatize, 'v')
# [token.text for token in myFilter3("We are going to do Text Analysis with whoosh.analysis")]

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): 
Mean Average Precision (MAP)

### Q1 (b): 
MAP considers the average pricision for all querys, and it takes into account the order of the relevant documents retrieved.

## Question 2

### Q2 (a): Write your code below

In [7]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q2, your query parser in QP_Q2, and your searcher in SEARCHER_Q2

In [8]:
# first, define a Schema for the index
mySchema = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

In [9]:
# if index exists - remove it
if os.path.isdir(INDEX_DIR):
    shutil.rmtree(INDEX_DIR)

# create the directory for the index
os.makedirs(INDEX_DIR)

# create index
myIndex = index.create_in(INDEX_DIR, mySchema)

In [10]:
# First, lets review the documents in our sample dataset
!dir $DOCUMENTS_DIR

 Volume in drive C is OS_Install
 Volume Serial Number is C015-BEDB

 Directory of C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\government\documents

2017-10-04  12:52 AM    <DIR>          .
2017-10-04  12:52 AM    <DIR>          ..
2017-09-25  12:37 PM            10,244 .DS_Store
2017-10-04  12:52 AM    <DIR>          00
2017-10-04  12:52 AM    <DIR>          01
2017-10-04  12:52 AM    <DIR>          02
2017-10-04  12:52 AM    <DIR>          03
2017-10-04  12:52 AM    <DIR>          04
2017-10-04  12:52 AM    <DIR>          05
2017-10-07  04:29 PM    <DIR>          06
2017-10-04  12:52 AM    <DIR>          07
2017-10-04  12:52 AM    <DIR>          08
2017-10-04  12:52 AM    <DIR>          09
2017-10-04  12:52 AM    <DIR>          10
2017-10-04  12:52 AM    <DIR>          11
2017-10-04  12:52 AM    <DIR>          12
2017-10-04  12:52 AM    <DIR>          13
2017-10-04  12:52 AM    <DIR>          14
2017-10-04  12:52 AM    <DIR>          15
2017-10-04  12:52 AM    <DIR>          16
2017

In [11]:
## Start Indexing
# first we build a list of all the full paths of the files in DOCUMENTS_DIR
filesToIndex1 = []
for root, dirs, files in os.walk(DOCUMENTS_DIR):
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    filesToIndex1.extend(filePaths)

In [12]:
# count files to index
print("number of files:", len(filesToIndex1))

number of files: 4078


In [13]:
# open writer
myWriter = writing.BufferedWriter(myIndex, period=20, limit=1000)

try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex1):
        with open(filePath, "r", encoding="utf-8" ) as f:
            fileContent = f.read()
            myWriter.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")

finally:
    # save the index
    myWriter.close()

already indexed: 1
already indexed: 1001
already indexed: 2001
already indexed: 3001
already indexed: 4001
done indexing.


In [14]:
# define a query parser for the field "file_content" in the index
# parser to define 1. content to search 2.shcema
myQueryParser = QueryParser("file_content", schema=myIndex.schema)
#searcher object
mySearcher = myIndex.searcher()

In [15]:
# print the topic file
!cat $QUER_FILE


'cat' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
# print the first 10 lines in the qrels file
!cat $QRELS_FILE

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()

# create an output file to which we'll write our results
outputTRECFile = open(OUTPUT_FILE, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1))
    topicQuery = myQueryParser.parse(topic_phrase)
    topicResults = mySearcher.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile.close()
topicsFile.close()

In [18]:
!$TREC_EVAL -q $QRELS_FILE $OUTPUT_FILE

num_ret               	1	1
num_rel               	1	5
num_rel_ret           	1	0
map                   	1	0.0000
Rprec                 	1	0.0000
bpref                 	1	0.0000
recip_rank            	1	0.0000
iprec_at_recall_0.00  	1	0.0000
iprec_at_recall_0.10  	1	0.0000
iprec_at_recall_0.20  	1	0.0000
iprec_at_recall_0.30  	1	0.0000
iprec_at_recall_0.40  	1	0.0000
iprec_at_recall_0.50  	1	0.0000
iprec_at_recall_0.60  	1	0.0000
iprec_at_recall_0.70  	1	0.0000
iprec_at_recall_0.80  	1	0.0000
iprec_at_recall_0.90  	1	0.0000
iprec_at_recall_1.00  	1	0.0000
P_5                   	1	0.0000
P_10                  	1	0.0000
P_15                  	1	0.0000
P_20                  	1	0.0000
P_30                  	1	0.0000
P_100                 	1	0.0000
P_200                 	1	0.0000
P_500                 	1	0.0000
P_1000                	1	0.0000
num_ret               	10	16
num_rel               	10	1
num_rel_ret           	10	1
map                   	10	0.1667
Rprec                 	10	0.0000


In [19]:
# Is it empty?
print("Index is empty?", myIndex.is_empty())


# How many files indexed?
print("Number of indexed files:", myIndex.doc_count())

Index is empty? False
Number of indexed files: 4078


In [20]:
INDEX_Q2 = myIndex # Replace None with your index for Q2
QP_Q2 = myQueryParser # Replace None with your query parser for Q2
SEARCHER_Q2 = mySearcher # Replace None with your searcher for Q2

### Q2 (b):
MAP_ALL = 0.1971

### Q2 (c): 
#### Very well:
ID: 18;24
<br>Value: 1
#### Very badly:
ID: 1; 2; 6; 7; 9; 16; 28
<br>Value: 0

## Question 3

### Q3 (a): 
**Query to analyze: 14 Agricultural biotechnology**
- False Negative: According to the Qrel file, G00-89-0000000 is the only relevant file to the query #14. 
In the baseline Whoosh system, it was retrieved with a rank of 4, while the top 3 result are all inrelevant. By    looking into the document, we can find words "Agriculture","agriculture", "agricultural" have the similar meaning but are not counted.
- False Positive: In the baseline Whoosh system, G00-09-1193469 was ranked the 2nd place, but it's not relevant to the query. By looking into the document, it's found that although the word "Agricultural" appeared multiple times, "biotechnology" only once. 

#### Improve:
- Weight less the word with higher document frequency.
- Add stemming, lower-case and stopfilter. So that the search is case-insensitive, more general and ignore the common words.
- Regularize the document length.
- Improve the weight if the words in a query are adjecent.

### Q3 (b): Write your code below

### Code below replaces the RegexTokenizer() with stmLwrStpIntraAnalyzer

In [21]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3

In [22]:
mySchema2 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = stmLwrStpIntraAnalyzer))

In [23]:
# if index exists - remove it
if os.path.isdir(INDEX_DIR2):
    shutil.rmtree(INDEX_DIR2)

# create the directory for the index
os.makedirs(INDEX_DIR2)

# create index or open it if already exists
myIndex2 = index.create_in(INDEX_DIR2, mySchema2)

In [24]:
## Start Indexing
# first we build a list of all the full paths of the files in DOCUMENTS_DIR
filesToIndex2 = []
for root, dirs, files in os.walk(DOCUMENTS_DIR):
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    filesToIndex2.extend(filePaths)
    
# open writer
myWriter2 = writing.BufferedWriter(myIndex2, period=20, limit=1000)

try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex2):
        with open(filePath, "r", encoding="utf-8" ) as f:
            fileContent = f.read()
            myWriter2.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")

finally:
    # save the index
    myWriter2.close()

already indexed: 1
already indexed: 1001
already indexed: 2001
already indexed: 3001
already indexed: 4001
done indexing.


In [25]:
# define a query parser for the field "file_content" in the index
myQueryParser2 = QueryParser("file_content", schema=myIndex2.schema)
mySearcher2 = myIndex2.searcher()

In [26]:
# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()

# create an output file to which we'll write our results
outputTRECFile2 = open(OUTPUT_FILE2, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1))
    topicQuery = myQueryParser2.parse(topic_phrase)
    topicResults = mySearcher2.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile2.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile2.close()
topicsFile.close()

In [27]:
!$TREC_EVAL -q $QRELS_FILE $OUTPUT_FILE2

num_ret               	1	3
num_rel               	1	5
num_rel_ret           	1	0
map                   	1	0.0000
Rprec                 	1	0.0000
bpref                 	1	0.0000
recip_rank            	1	0.0000
iprec_at_recall_0.00  	1	0.0000
iprec_at_recall_0.10  	1	0.0000
iprec_at_recall_0.20  	1	0.0000
iprec_at_recall_0.30  	1	0.0000
iprec_at_recall_0.40  	1	0.0000
iprec_at_recall_0.50  	1	0.0000
iprec_at_recall_0.60  	1	0.0000
iprec_at_recall_0.70  	1	0.0000
iprec_at_recall_0.80  	1	0.0000
iprec_at_recall_0.90  	1	0.0000
iprec_at_recall_1.00  	1	0.0000
P_5                   	1	0.0000
P_10                  	1	0.0000
P_15                  	1	0.0000
P_20                  	1	0.0000
P_30                  	1	0.0000
P_100                 	1	0.0000
P_200                 	1	0.0000
P_500                 	1	0.0000
P_1000                	1	0.0000
num_ret               	10	42
num_rel               	10	1
num_rel_ret           	10	1
map                   	10	0.2500
Rprec                 	10	0.0000


In [28]:
INDEX_Q3 = myIndex2  # Replace None with your index for Q3
QP_Q3 = myQueryParser2 # Replace None with your query parser for Q3
SEARCHER_Q3 = mySearcher2 # Replace None with your searcher for Q3

**Comparsion between search performance on query 14**

In [29]:
'''
Extract topic # 14
'''
test_query = None
with open(QUER_FILE, 'r') as f:
    topics = f.read().splitlines()
    test_query=topics[7].split(' ',1)[1]
    print(test_query)

Agricultural biotechnology


In [30]:
# tokenizing query with regex
[token.text for token in tokenizer("Agricultural biotechnology")]

['Agricultural', 'biotechnology']

In [31]:
# tokenizing query with improved analyzer
[token.text for token in stmLwrStpIntraAnalyzer("Agricultural biotechnology")]

['agricultur', 'biotechnolog']

In [32]:
'''
Show search result with Vanilla Whoosh for query 14.
'''
sampleQuery = myQueryParser.parse("Agricultural biotechnology")
sampleQueryResults = mySearcher.search(sampleQuery, limit=None)

for (docnum, result) in enumerate(sampleQueryResults):
    score = sampleQueryResults.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)

G00-79-4144643 0 16.017276887886855
G00-09-1193469 1 15.942972766466898
G00-45-0809730 2 15.640292921668705
G00-89-0000000 3 15.377247319282251
G00-88-1894712 4 8.188481344826137
G00-34-0444524 5 3.239980601052612
G00-84-0274223 6 1.544028703546812


In [33]:
'''
Show search result with improved Whoosh for query 14.
'''
sampleQuery2 = myQueryParser2.parse("Agricultural biotechnology")
# run searcher with parse object
sampleQueryResults2 = mySearcher2.search(sampleQuery2, limit=None)

# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults2):
    score = sampleQueryResults2.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)

G00-89-0000000 0 18.403093246203667
G00-79-4144643 1 16.301717688872152
G00-01-3251318 2 15.722688660323271
G00-97-1475424 3 15.722688660323271
G00-51-1924264 4 15.532060347556094
G00-72-0385489 5 15.192137168081594
G00-86-2926152 6 14.2070816709651
G00-45-0809730 7 14.050786774335535
G00-09-1193469 8 13.915901570783479
G00-36-2788975 9 13.641657061547104
G00-97-2215955 10 13.449700178400025
G00-09-1404763 11 13.329916769872067
G00-10-2024294 12 13.17293901116626
G00-91-1609512 13 13.141921398598672
G00-06-2853218 14 12.522058301150393
G00-35-2527252 15 12.453188219641742
G00-06-0690672 16 12.203635726283462
G00-86-0847220 17 11.473727625967161
G00-88-1894712 18 11.348435559086619
G00-70-3424520 19 10.979337567864256
G00-07-2371962 20 9.246401803737616
G00-34-0444524 21 6.383054619942959
G00-21-4119651 22 5.572173996767903
G00-84-0274223 23 5.000341059795548
G00-82-3144058 24 2.0191837223306757
G00-46-0840102 25 1.8474602852606614


### Q3 (c): 
####     Modification:
- RegexTokenizer(), the analyzer used in vanilla Whoosh, is replaced with stmLwrStpIntraAnalyzer. That change makes the search case-insensitive, ignore stop words and stems the words.
#### Improvement:
- The overall MAP score is increased from 0.1971 to 0.3366, achieves an improvement of 70%. It also greatly improves individual query performance.
- The MAP for Query 14 improved from 0.25 to 1.0. The False Negative case in Q2 was corrected, it's rank improved from the 4th to the 1st. The False Positive case in Q2 was also corrected, it's rank droped from the 2nd to the 9th.

### Q3 (d): 
Yes

### Q3 (e): 
Yes

### Q3 (f): 
I think it's a good idea.
- It increase the overall MAP performance by 70%.
- The performance of 7 topics got improved greatly and only 2 topics' performace were slightly decreased.

## Question 4 (Graduate Students)

In [34]:
GRAD_STUDENT = True # change to True if you are a grad student

### Q4 (a):  
#### As the score of query in Q3(a) is already optimized, we will look into another query here.
**Query to analyze: 9 genealogy searches**
- False Negative: According to the Qrel file, G00-91-3181951  is the only relevant file to the query #9. 
In the baseline Whoosh system, it was retrieved with a rank of 13, while the top 12 result are all inrelevant. By looking into the document, we can find both the words genealoy and searches appears multiple times, but the overall length of the document is much longer than the other higher ranked document. The score is low because its length is normalized too much.
- False Positive: In the baseline Whoosh system, G00-06-1975174 was ranked the 4th, but it's not relevant to the query. By looking into the document, it's found that it's a catalogue file and the word combination "geneology searches" appeared only once. The length of the document is very short, which means it's may have a better score after length normalization.

#### Improve:
- Avoid regularizing document length too much.

### Q4 (b): Write your code below

In [35]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q4, your query parser in QP_Q4, and your searcher in SEARCHER_Q4

In [36]:
mySchema3 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = stmLwrStpIntraAnalyzer))

In [37]:
# if index exists - remove it
if os.path.isdir(INDEX_DIR3):
    shutil.rmtree(INDEX_DIR3)

# create the directory for the index
os.makedirs(INDEX_DIR3)

# create index or open it if already exists
myIndex3 = index.create_in(INDEX_DIR3, mySchema3)

In [38]:
## Start Indexing
# first we build a list of all the full paths of the files in DOCUMENTS_DIR
filesToIndex3 = []
for root, dirs, files in os.walk(DOCUMENTS_DIR):
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    filesToIndex3.extend(filePaths)


# open writer
myWriter3 = writing.BufferedWriter(myIndex3, period=20, limit=1000)

try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex3):
        with open(filePath, "r", encoding="utf-8" ) as f:
            fileContent = f.read()
            myWriter3.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")

finally:
    # save the index
    myWriter3.close()

already indexed: 1
already indexed: 1001
already indexed: 2001
already indexed: 3001
already indexed: 4001
done indexing.


In [39]:
# define a query parser for the field "file_content" in the index
# B: proportional to level of length normalizaiton; 
# K1: sensitivity to term-frequency
myQueryParser3 = QueryParser("file_content", schema=myIndex3.schema)
mySearcher3 = myIndex3.searcher(weighting = scoring.BM25F(B=0.33, K1=1.2))


# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()

# create an output file to which we'll write our results
outputTRECFile3 = open(OUTPUT_FILE3, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1))
    topicQuery = myQueryParser3.parse(topic_phrase)
    topicResults = mySearcher3.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile3.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile3.close()
topicsFile.close()

!$TREC_EVAL -q $QRELS_FILE $OUTPUT_FILE3

num_ret               	1	3
num_rel               	1	5
num_rel_ret           	1	0
map                   	1	0.0000
Rprec                 	1	0.0000
bpref                 	1	0.0000
recip_rank            	1	0.0000
iprec_at_recall_0.00  	1	0.0000
iprec_at_recall_0.10  	1	0.0000
iprec_at_recall_0.20  	1	0.0000
iprec_at_recall_0.30  	1	0.0000
iprec_at_recall_0.40  	1	0.0000
iprec_at_recall_0.50  	1	0.0000
iprec_at_recall_0.60  	1	0.0000
iprec_at_recall_0.70  	1	0.0000
iprec_at_recall_0.80  	1	0.0000
iprec_at_recall_0.90  	1	0.0000
iprec_at_recall_1.00  	1	0.0000
P_5                   	1	0.0000
P_10                  	1	0.0000
P_15                  	1	0.0000
P_20                  	1	0.0000
P_30                  	1	0.0000
P_100                 	1	0.0000
P_200                 	1	0.0000
P_500                 	1	0.0000
P_1000                	1	0.0000
num_ret               	10	42
num_rel               	10	1
num_rel_ret           	10	1
map                   	10	0.3333
Rprec                 	10	0.0000


In [40]:
INDEX_Q4 = myIndex3 # Replace None with your index for Q4
QP_Q4 = myQueryParser3 # Replace None with your query parser for Q4
SEARCHER_Q4 = mySearcher3 # Replace None with your searcher for Q4

**Comparion of performences on topic 9.**

In [41]:
# tokenizing query with regex
[token.text for token in stmLwrStpIntraAnalyzer("genealogy searches")]

['genealog', 'search']

In [42]:
'''
Show search result with vanilla Whoosh for query 9.
'''
sampleQuery29 = myQueryParser3.parse("genealogy searches")
# run searcher with parse object
sampleQueryResults29 = mySearcher2.search(sampleQuery29, limit=None)

# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults29):
    score = sampleQueryResults29.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)

G00-30-0221651 0 14.21403104174161
G00-79-2892445 1 13.432220968622268
G00-26-1048210 2 12.400303034907044
G00-06-1975174 3 10.412381648452014
G00-59-0523165 4 10.412381648452014
G00-88-2629440 5 10.412381648452014
G00-95-3755341 6 10.412381648452014
G00-24-0016657 7 10.362376642595706
G00-95-3337324 8 10.362376642595706
G00-01-2134408 9 10.18979815087839
G00-33-1729611 10 10.076429644350592
G00-01-2898660 11 9.887548802267633
G00-91-3181951 12 9.817399027993426
G00-43-3812747 13 9.750866265136219
G00-21-1529615 14 9.441772117492475
G00-55-0643570 15 9.441772117492475
G00-67-1176122 16 9.153508165890496
G00-08-3780534 17 9.100261804513455
G00-49-2630728 18 8.609149939392754
G00-02-1372443 19 8.577008166061567
G00-08-0900666 20 8.577008166061567
G00-08-1314254 21 8.577008166061567
G00-00-2016453 22 8.548039950210205
G00-59-3622783 23 6.540653073970914
G00-48-2464830 24 6.144316770561288
G00-22-3171818 25 6.035626932457669
G00-21-2737022 26 4.567961775290067


In [43]:
'''
Show search result with Q4 improved Whoosh for query 9.
'''
sampleQuery39 = myQueryParser3.parse("genealogy searches")
# run searcher with parse object
sampleQueryResults39 = mySearcher3.search(sampleQuery39, limit=None)

# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults39):
    score = sampleQueryResults39.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)

G00-79-2892445 0 14.568173151201915
G00-30-0221651 1 13.843207399936215
G00-26-1048210 2 12.357541257435114
G00-91-3181951 3 11.840513330199983
G00-43-3812747 4 10.38558052682346
G00-24-0016657 5 9.374541895812964
G00-95-3337324 6 9.374541895812964
G00-01-2134408 7 9.312685875111313
G00-01-2898660 8 8.979015025237526
G00-59-3622783 9 8.919607734827501
G00-06-1975174 10 8.70020516264145
G00-59-0523165 11 8.70020516264145
G00-88-2629440 12 8.70020516264145
G00-95-3755341 13 8.70020516264145
G00-08-3780534 14 8.680160660073854
G00-49-2630728 15 8.675876088981617
G00-33-1729611 16 8.594853563209137
G00-02-1372443 17 8.463796148963468
G00-08-0900666 18 8.463796148963468
G00-08-1314254 19 8.463796148963468
G00-21-1529615 20 8.383364102073283
G00-55-0643570 21 8.383364102073283
G00-67-1176122 22 8.28147495051222
G00-00-2016453 23 8.054368689288513
G00-48-2464830 24 7.443324828441014
G00-22-3171818 25 6.868938660995397
G00-21-2737022 26 5.91696779903032


### Q4 (c): 
**Modification:**
While the k1 parameter is kept unchanged, the b parameter is reduced from 0.75 to 0.33. So that the scoring system has less normalization effect on document length.

<br>**Improvement**
Compared with result of Q3
- The overall MAP score is increaed from 0.3366 to 0.3587. 
- The false negative case in 4(a) now has a higher rank of the 4th compared with the previous rank of the 13th. 
- The false positive case in 4(a) now has a lower rank of the 11th compared with the previous rank of the 4nd.

### Q4 (d): 
Yes

### Q4 (e): 
Yes

### Q4 (f): 
It is a good idea. The new scoring method has improved the overall performance compared with both in Q3 and Vanilla Whoosh. The MAP score on query 9 and 10 is significantly improved. While the MAP score on query 22 and 26 is reduced by a negligibly small amount.

## Validation

In [44]:
# Run the following cells to make sure your code returns the correct value types

In [45]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Path Validation

In [46]:
assert "MATERIALS_DIR" in globals(), "variable MATERIALS_DIR does not exists"
assert(os.path.isdir(os.path.join(MATERIALS_DIR))), "MATERIALS_DIR folder does not exists"
assert(os.path.isdir(os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2"))), "invalid folder structure"
assert(os.path.isdir(os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\government\documents"))), "invalid folder structure"
print("Paths validated")

Paths validated


### Q2 Validation

In [47]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [48]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

Q3 Types Validated


### Q4 Validation (Graduate Students)

In [49]:
assert((not GRAD_STUDENT) or isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert((not GRAD_STUDENT) or isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert((not GRAD_STUDENT) or isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
